# Load all the necessary libraries, these should already be installed.

### If you need additional libraries, add them to the dependencies.r files and rebuild the image


In [1]:
library(knitr)
library(plotly)
library(httr)
library(jsonlite)
library(purrr)
library(magrittr)
library(GetoptLong)

Loading required package: ggplot2

Attaching package: ‘plotly’

The following object is masked from ‘package:ggplot2’:

    last_plot

The following object is masked from ‘package:stats’:

    filter

The following object is masked from ‘package:graphics’:

    layout


Attaching package: ‘httr’

The following object is masked from ‘package:plotly’:

    config


Attaching package: ‘purrr’

The following object is masked from ‘package:jsonlite’:

    flatten


Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names



# Functions that will eventually be in an R library and maintained alongside the API

### This is PIC-SURE-HPDS specific code, the same functionality is available through PIC-SURE 2's HPDS resource, but this saves you the PIC-SURE ceremonial wrapping of JSON in more JSON.


In [2]:

escapeSlashes <- function(field){
    return (gsub("\\", "\\\\", field, fixed=TRUE))
}

wrapInQuotes <- function(value){
    return (paste('"', escapeSlashes(value), '"', sep=''))
}

numericFilter <- function(field, min, max){
    field <- escapeSlashes(field)
    return (qq('"@{field}":{"min":@{min},"max":@{max}}'))
}

categoryFilter <- function(field, categories){
    field <- escapeSlashes(field)
    categories <- paste(map(categories, wrapInQuotes), collapse=', ')
    return (qq('"@{field}":[@{categories}]'))
}


dictionary <- function(query){
    response <- POST("http://pic-sure-hpds-nhanes:8080/PIC-SURE/search/", body=paste('{"query":"',query,'"}', sep=""), content_type_json())
    response
    mergeCategoryValues <- function(entry){
        entry$categoryValues
        entry$categoryValues <- paste(entry$categoryValues, collapse=", ")
        return (entry)        
    }
    
    return(map_df(content(response)$results, mergeCategoryValues))
}    

dictionaryFields <- function(query){
    as.list(dictionary(query)$name)
}

query <- function(selectedFields = NULL, requiredFields = NULL, numericFilters = NULL, categoryFilters = NULL) {
    fields <- function(selectedFields){
        paste(map(selectedFields, wrapInQuotes), collapse=', ')
    }

    query <- qq(
        '{"fields":[@{fields}],
        "requiredFields":[@{requiredFields}],
        "numericFilters":{@{numericFilters}},
        "categoryFilters":{@{categoryFilters}}}', 
          envir = list(
              fields = fields(selectedFields), 
              requiredFields = fields(requiredFields), 
              numericFilters = paste(numericFilters, sep=","), 
              categoryFilters = paste(categoryFilters, sep=",")))
    query
    response <- POST("http://pic-sure-hpds-nhanes:8080/query", body=query, content_type_json())
    content(response)
}

getStatus <- function(queryId){
    response <- GET(paste("http://pic-sure-hpds-nhanes:8080/query", queryId, "status", sep="/"))
    content(response)
}

getResult <- function(queryId){
    return(read.csv(paste("http://pic-sure-hpds-nhanes:8080/query", queryId, "result", sep="/")))
}

runQueryAndGetResult <- function(selectedFields = NULL, requiredFields = NULL, numericFilters = NULL, categoryFilters = NULL){
    queryStatus <- query(selectedFields, requiredFields, numericFilters, categoryFilters)
    while(queryStatus$status == "RUNNING" || queryStatus$status == "PENDING"){
        Sys.sleep(.1)
        queryStatus <- getStatus(queryStatus$id)
    }
    if(queryStatus$status == "SUCCESS" && queryStatus$numRows > 0){
        print(paste(
            queryStatus$numRows, " rows and ", 
            queryStatus$numColumns, " columns ", 
            "completed in : ", (queryStatus$completedTime - queryStatus$queuedTime)/1000, " seconds", sep=""))
        return (getResult(queryStatus$id))
    }else{
        queryStatus
    }
}


# Let's start by looking at the whole dictionary file

## Notice that the important data is all here, if it's continuous we get min and max values, if it's categorical we get all the available categories for filtering. We also get a count of observations, at this point this is a patient count only because we haven't dealt with temporal relationships yet.

In [3]:
dictionary("demographics")

,categorical,observationCount,max,min,name,categoryValues
1,FALSE,41474,4,1,\demographics\SDDSRVYR\,
2,FALSE,38076,5,0,\demographics\INDFMPIR\,
3,FALSE,38076,2,0,\demographics\SES_LEVEL\,
4,TRUE,41474,NA,NA,\demographics\SEX\,"female, male"
5,FALSE,41445,3,1,\demographics\DMDBORN\,
6,FALSE,21004,103831.17,0,\demographics\WTMEC4YR\,
7,FALSE,26793,99,1,\demographics\DMDMARTL\,
8,FALSE,41474,85,0,\demographics\AGE\,
9,FALSE,41474,3,1,\demographics\SDMVPSU\,
10,TRUE,41474,NA,NA,\demographics\RACE\,"black, mexican, other, other_hispanic, white"


# Let's look at something more interesting... all variables related to "blood"


In [4]:
dictionary("blood")

,categorical,observationCount,max,min,name,categoryValues
1,FALSE,4273,0.7071,0.0707,\laboratory\volatile compounds\Blood Nitrobenezene (ng per mL)\,
2,FALSE,5369,6270,0.1344,\laboratory\volatile compounds\Blood MTBE (pg per mL)\,
3,FALSE,4486,0.0679,0.0354,"\laboratory\volatile compounds\Blood 1,3-Dichlorobenzene (ng per mL)\",
4,FALSE,33604,5.3,0,\laboratory\blood\Eosinophils number\,
5,FALSE,4047,18.7,0.0495,\laboratory\volatile compounds\Blood Methylene Chloride (ng per mL)\,
6,FALSE,4435,0.056,0.0064,"\laboratory\volatile compounds\Blood 1,2-Dichloroethane (ng per mL)\",
7,FALSE,5616,249,0.026,"\laboratory\volatile compounds\Blood 1,4-Dichlorobenzene (ng per mL)\",
8,FALSE,5573,0.747,0.0035,\laboratory\volatile compounds\Blood Carbon Tetrachloride (ng per mL)\,
9,FALSE,33718,19.7,5.8,\laboratory\blood\Hemoglobin (g per dL)\,
10,FALSE,26036,132,0,\examination\blood pressure\mean diastolic\,


# OK, so how about all that data for "blood"? 

### Notice here we just pass the result of our dictionary search, but by using the convenient dictionaryFields function that handles escaping our terms for us and building a vector of fields. 

In [5]:

data = runQueryAndGetResult(dictionaryFields("blood"))

[1] "41474 rows and 58 columns completed in : 1.22 seconds"


In [6]:
data

,Patient.ID,X.examination.blood.pressure.60.sec.HR..30.sec.HR...2..,X.examination.blood.pressure.mean.diastolic.,X.examination.blood.pressure.mean.systolic.,X.laboratory.biochemistry.Blood.urea.nitrogen..mg.per.dL..,X.laboratory.blood.Basophils.number.,X.laboratory.blood.Basophils.percent.....,X.laboratory.blood.Eosinophils.number.,X.laboratory.blood.Eosinophils.percent.....,X.laboratory.blood.Hematocrit.....,⋯,X.laboratory.volatile.compounds.Blood.Nitrobenezene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.Styrene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.Tetrachloroethene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.Toluene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.Trichloroethene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.cis.1.2.Dichloroethene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.m..p.Xylene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.o.Xylene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.trans.1.2.Dichloroethene..ng.per.mL..,X.questionnaire.disease.blood_cancer_self_report.
1,1,NaN,78.66667,124,9,0,0.4,0.4,11,39.8,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
2,2,NaN,NaN,NaN,12,0,0.2,0.2,2.1,42.4,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
3,3,126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,4,NaN,74,118,12,0.1,1.2,0.2,3.7,44.7,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
5,5,NaN,30,96,14,0,0.3,0.2,2,43,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
6,6,106,NaN,NaN,NaN,0,0.5,0.1,2.3,38.5,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
7,7,NaN,NaN,NaN,15,0.1,0.6,0.1,1,46,⋯,0.2121,0.038,0.0339,0.076,0.0085,0.0071,0.12,0.032,0.0071,No
8,8,NaN,77.33333,106,18,0,0.4,0.3,3.8,44.1,⋯,NaN,NaN,0.0146,0.249,0.0068,NaN,0.0845,0.0215,NaN,No
9,9,NaN,87.33333,131.33333,9,0.1,1,0.3,4.1,42.8,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
10,10,NaN,NaN,NaN,14,0,0.6,0.1,1.3,46.7,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No


# OK, so how about all that data for "blood" but only where patients have a Basophils percent? 

### Notice we are requiring patients to have all fields that match the search "Basophils percent". If we try this for the search term "blood" it will require all fields that have blood in their path, which matches no patients so we would get no results.

In [7]:
data = runQueryAndGetResult(selectedFields=dictionaryFields("blood"), 
                     requiredFields=dictionaryFields("Basophils percent"))


[1] "33605 rows and 58 columns completed in : 0.53 seconds"


In [8]:
data

,Patient.ID,X.examination.blood.pressure.60.sec.HR..30.sec.HR...2..,X.examination.blood.pressure.mean.diastolic.,X.examination.blood.pressure.mean.systolic.,X.laboratory.biochemistry.Blood.urea.nitrogen..mg.per.dL..,X.laboratory.blood.Basophils.number.,X.laboratory.blood.Basophils.percent.....,X.laboratory.blood.Eosinophils.number.,X.laboratory.blood.Eosinophils.percent.....,X.laboratory.blood.Hematocrit.....,⋯,X.laboratory.volatile.compounds.Blood.Nitrobenezene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.Styrene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.Tetrachloroethene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.Toluene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.Trichloroethene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.cis.1.2.Dichloroethene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.m..p.Xylene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.o.Xylene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.trans.1.2.Dichloroethene..ng.per.mL..,X.questionnaire.disease.blood_cancer_self_report.
1,1,NaN,78.66667,124,9,0,0.4,0.4,11,39.8,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
2,2,NaN,NaN,NaN,12,0,0.2,0.2,2.1,42.4,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
3,4,NaN,74,118,12,0.1,1.2,0.2,3.7,44.7,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
4,5,NaN,30,96,14,0,0.3,0.2,2,43,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
5,6,106,NaN,NaN,NaN,0,0.5,0.1,2.3,38.5,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
6,7,NaN,NaN,NaN,15,0.1,0.6,0.1,1,46,⋯,0.2121,0.038,0.0339,0.076,0.0085,0.0071,0.12,0.032,0.0071,No
7,8,NaN,77.33333,106,18,0,0.4,0.3,3.8,44.1,⋯,NaN,NaN,0.0146,0.249,0.0068,NaN,0.0845,0.0215,NaN,No
8,9,NaN,87.33333,131.33333,9,0.1,1,0.3,4.1,42.8,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
9,10,NaN,NaN,NaN,14,0,0.6,0.1,1.3,46.7,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
10,11,NaN,47.33333,115.33333,7,0,0.3,0.1,1.3,33.1,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


# Right... 33605 patients, let's try the same query, but only age 5-25

In [9]:
numFilters <- c(numericFilter("\\demographics\\AGE\\",5,25))
data = runQueryAndGetResult(selectedFields=dictionaryFields("blood"), 
                     requiredFields=dictionaryFields("Basophils percent"), 
                     numericFilters=numFilters)

[1] "14542 rows and 59 columns completed in : 3.007 seconds"


In [10]:
data

,Patient.ID,X.demographics.AGE.,X.examination.blood.pressure.60.sec.HR..30.sec.HR...2..,X.examination.blood.pressure.mean.diastolic.,X.examination.blood.pressure.mean.systolic.,X.laboratory.biochemistry.Blood.urea.nitrogen..mg.per.dL..,X.laboratory.blood.Basophils.number.,X.laboratory.blood.Basophils.percent.....,X.laboratory.blood.Eosinophils.number.,X.laboratory.blood.Eosinophils.percent.....,⋯,X.laboratory.volatile.compounds.Blood.Nitrobenezene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.Styrene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.Tetrachloroethene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.Toluene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.Trichloroethene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.cis.1.2.Dichloroethene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.m..p.Xylene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.o.Xylene..ng.per.mL..,X.laboratory.volatile.compounds.Blood.trans.1.2.Dichloroethene..ng.per.mL..,X.questionnaire.disease.blood_cancer_self_report.
1,5,18,NaN,30,96,14,0,0.3,0.2,2,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,11,17,NaN,47.33333,115.33333,7,0,0.3,0.1,1.3,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,19,12,NaN,48.66667,110,5,0,0.7,0.1,1.4,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,22,20,NaN,51,100,13,0,0.7,0.1,2.3,⋯,NaN,0.045,0.0332,0.074,0.0085,NaN,0.077,0.0346,NaN,No
5,25,18,NaN,60.66667,96.66667,16,0,0.6,0.1,0.9,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
6,26,23,NaN,72.66667,117.33333,24,0.1,0.8,0.1,0.9,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
7,29,17,NaN,71,124,17,0.1,0.6,0.7,8.4,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
8,30,19,NaN,68,108,13,0,0.5,0.1,1.6,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
9,33,12,NaN,44,78,9,0.1,0.8,1,11.4,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
10,38,19,NaN,64.66667,115.33333,5,0,0.4,0.2,2,⋯,0.2121,0.0212,0.0339,0.073,0.0085,0.0071,0.083,0.017,0.0071,


# This is better, what about only Male patients who are of white or mexican decent and we want to add all lab values?

In [11]:
numFilters <- c(numericFilter("\\demographics\\AGE\\",5,25))
catFilters <- c(categoryFilter("\\demographics\\RACE\\", c("white", "mexican")))
data = runQueryAndGetResult(selectedFields=union(dictionaryFields("laboratory"), dictionaryFields("blood")), 
                     requiredFields=dictionaryFields("Basophils percent"), 
                     numericFilters=numFilters, categoryFilters=catFilters)

[1] "8870 rows and 502 columns completed in : 12.761 seconds"


In [12]:
data

,Patient.ID,X.demographics.AGE.,X.demographics.RACE.,X.examination.blood.pressure.60.sec.HR..30.sec.HR...2..,X.examination.blood.pressure.mean.diastolic.,X.examination.blood.pressure.mean.systolic.,X.laboratory.acrylamide.,X.laboratory.acrylamide.Acrylamide..pmoL.per.G.Hb..,X.laboratory.acrylamide.Glycideamide..pmoL.per.G.Hb..,X.laboratory.aging.Mean.Telomere.Length.,⋯,X.laboratory.volatile.compounds.Toluene..ug.per.cubic.meter..,X.laboratory.volatile.compounds.Trichloroethene..ug.per.cubic.meter..,X.laboratory.volatile.compounds.Water.Bromodichloromethane..ng.per.mL..,X.laboratory.volatile.compounds.Water.Bromoform..ng.per.mL..,X.laboratory.volatile.compounds.Water.Chloroform..ng.per.mL..,X.laboratory.volatile.compounds.Water.Dibromochloromethane..ng.per.mL..,X.laboratory.volatile.compounds.Water.MTBE..ng.per.mL..,X.laboratory.volatile.compounds.m.p.Xylene..ug.per.cubic.meter..,X.laboratory.volatile.compounds.o.Xylene..ug.per.cubic.meter..,X.questionnaire.disease.blood_cancer_self_report.
1,5,18,mexican,NaN,30,96,No,77.6,74.3,NaN,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,19,12,mexican,NaN,48.66667,110,No,NaN,NaN,NaN,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,22,20,mexican,NaN,51,100,No,NaN,NaN,1.13203,⋯,NaN,NaN,1.6,0.0707,1.6,0.36,0.0707,NaN,NaN,No
4,25,18,mexican,NaN,60.66667,96.66667,No,NaN,NaN,NaN,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
5,26,23,mexican,NaN,72.66667,117.33333,No,NaN,NaN,0.97874,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
6,33,12,white,NaN,44,78,No,32.7,25.3,NaN,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
7,38,19,mexican,NaN,64.66667,115.33333,No,NaN,NaN,NaN,⋯,NaN,NaN,6.2,0.37,8.2,3.9,0.0707,NaN,NaN,
8,44,5,mexican,84,NaN,NaN,No,NaN,NaN,NaN,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
9,45,16,mexican,NaN,62.66667,115.33333,No,NaN,NaN,NaN,⋯,NaN,NaN,16,3.5,11,17,0.0707,NaN,NaN,
10,47,17,mexican,NaN,54,136.66667,No,NaN,NaN,NaN,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


# Now if we really just want all the data, we can do something like this

In [ ]:
data = runQueryAndGetResult(dictionaryFields(""))

In [ ]:
data